In [1]:
import sys, platform
import sherpa_onnx
import onnxruntime as ort

print("Python exe :", sys.executable)
print("Python ver :", sys.version.split()[0])
print("Platform   :", platform.platform(), "| Machine:", platform.machine())
print("sherpa_onnx:", getattr(sherpa_onnx, "__version__", "unknown"))
print("ORT providers:", ort.get_available_providers())

Python exe : /opt/homebrew/Caskroom/miniforge/base/envs/asr-korean/bin/python
Python ver : 3.11.13
Platform   : macOS-15.6.1-arm64-arm-64bit | Machine: arm64
sherpa_onnx: 1.12.10
ORT providers: ['CPUExecutionProvider']


In [2]:
from pathlib import Path

# 네가 실제로 사용 중인 모델 폴더
extract_dir = Path("/Users/leejeje/Desktop/DSL/25-1/Modeling/model/sherpa-onnx-streaming-zipformer-korean-2024-06-16")
tokens = extract_dir / "tokens.txt"

def pick(prefix: str):
    # int8이 있으면 우선 사용, 없으면 fp32 사용
    int8 = sorted(extract_dir.glob(f"{prefix}-epoch-*.int8.onnx"))
    fp32 = sorted(p for p in extract_dir.glob(f"{prefix}-epoch-*.onnx") if ".int8." not in p.name)
    return (int8[-1] if int8 else (fp32[-1] if fp32 else None))

ENC  = pick("encoder")
DEC  = pick("decoder")
JOIN = pick("joiner")

print("tokens:", tokens, tokens.exists())
print("encoder:", ENC)
print("decoder:", DEC)
print("joiner :", JOIN)

assert extract_dir.exists(), f"모델 폴더가 없습니다: {extract_dir}"
assert tokens.exists(), "tokens.txt가 없습니다."
assert ENC and DEC and JOIN, "encoder/decoder/joiner onnx 파일을 찾지 못했습니다."


tokens: /Users/leejeje/Desktop/DSL/25-1/Modeling/model/sherpa-onnx-streaming-zipformer-korean-2024-06-16/tokens.txt True
encoder: /Users/leejeje/Desktop/DSL/25-1/Modeling/model/sherpa-onnx-streaming-zipformer-korean-2024-06-16/encoder-epoch-99-avg-1.int8.onnx
decoder: /Users/leejeje/Desktop/DSL/25-1/Modeling/model/sherpa-onnx-streaming-zipformer-korean-2024-06-16/decoder-epoch-99-avg-1.int8.onnx
joiner : /Users/leejeje/Desktop/DSL/25-1/Modeling/model/sherpa-onnx-streaming-zipformer-korean-2024-06-16/joiner-epoch-99-avg-1.int8.onnx


In [3]:
import sherpa_onnx

recognizer = sherpa_onnx.OnlineRecognizer.from_transducer(
    tokens=str(tokens),
    encoder=str(ENC),
    decoder=str(DEC),
    joiner=str(JOIN),
    decoding_method="greedy_search",
    num_threads=2,
    provider="cpu",   # 가능하면 "coreml"도 시도해 볼 수 있음
)

print("✅ 모델 로드 완료")
print(" - tokens :", tokens.name)
print(" - encoder:", Path(ENC).name)
print(" - decoder:", Path(DEC).name)
print(" - joiner :", Path(JOIN).name)


✅ 모델 로드 완료
 - tokens : tokens.txt
 - encoder: encoder-epoch-99-avg-1.int8.onnx
 - decoder: decoder-epoch-99-avg-1.int8.onnx
 - joiner : joiner-epoch-99-avg-1.int8.onnx
